<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Tick-size-change-history" data-toc-modified-id="Tick-size-change-history-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Tick size change history</a></span><ul class="toc-item"><li><span><a href="#Processing-Technical-Information-for-5,-10,-15,-30,-45,-60-Minutes" data-toc-modified-id="Processing-Technical-Information-for-5,-10,-15,-30,-45,-60-Minutes-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Processing Technical Information for 5, 10, 15, 30, 45, 60 Minutes</a></span></li></ul></li><li><span><a href="#Votatilities" data-toc-modified-id="Votatilities-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Votatilities</a></span><ul class="toc-item"><li><span><a href="#Garman-Klass-Volatility" data-toc-modified-id="Garman-Klass-Volatility-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Garman Klass Volatility</a></span></li><li><span><a href="#Yang-Zhang-Volatility" data-toc-modified-id="Yang-Zhang-Volatility-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Yang Zhang Volatility</a></span></li><li><span><a href="#Processing-Volatilities-for-5,-10,-15,-30,-45,-60-Minutes" data-toc-modified-id="Processing-Volatilities-for-5,-10,-15,-30,-45,-60-Minutes-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Processing Volatilities for 5, 10, 15, 30, 45, 60 Minutes</a></span></li></ul></li></ul></div>

In [1]:
# Import Modules
import pandas as pd
import numpy as np
import math
import time
import datetime
import statistics

import os
import warnings
warnings.filterwarnings('ignore')

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics(ip))

<IPython.core.display.Javascript object>

# Tick size change history

Price (JPY) up to | 2008/07/22 | 2010/01/04 | 2014/01/14 | 2014/07/22 | 2015/09/24
----------------- | ---------- | ---------- | ---------- | ---------- | ----------
1,000  | 1 | 1 | 1 | 0.1 | 0.1  
2,000  | 1 | 1 | 1 | 0.5 | 0.5
3,000  | 5 | 1 | 1 | 0.5 | 0.5
5,000  | 10 | 5 | 1 | 0.5 | 1
10,000  | 10 | 10 | 1 | 1 | 1
30,000  | 10 | 10 | 5 | 5 | 5
50,000  | 50 | 50 | 5 | 5 | 10
100,000  | 100 | 100 | 10 | 10 | 10
300,000  | 100 | 100 | 50 | 50 | 50
500,000  | 1,000 | 500 | 50 | 50 | 100
1 million  | 1,000 | 1,000 | 100 | 100 | 100
3 million  | 1,000 | 1,000 | 500 | 500 | 500
5 million  | 10,000 | 5,000 | 500 | 500 | 1,000
10 million  | 10,000 | 10,000 | 1,000 | 1,000 | 1,000
20 million  | 10,000 | 10,000 | 5,000 | 5,000 | 5,000
30 million  | 50,000 | 10,000 | 5,000 | 5,000 | 5,000
50 million  | 100,000 | 50,000 | 5,000 | 5,000 | 10,000
over 50 million  | 100,000 | 100,000 | 10,000 | 10,000 | 10,000

In [ ]:
# list(set(df['a'])) = unique

In [2]:
def pre_process(filename):

    data = pd.read_csv(filename)[['Date','Time','Price','Trading volume']]

    hour=[];minute=[];sec=[];milsec=[]

    for i in range(len(data)):
        time_i = data.Time[i]
        hour.append(str(time_i)[:-7])
        minute.append(str(time_i)[-7:-5])
        sec.append(str(time_i)[-5:-3])
        milsec.append(str(time_i)[-3:])

    data = pd.DataFrame({
        'Date':data.Date,'Hour':hour, 'Minute':minute, 'Second':sec, 'Milsec':milsec, 
        'Price':data.Price, 
        'Volume':data['Trading volume'] })

    data.Hour = pd.to_numeric(data.Hour, errors='coerce')
    data.Minute = pd.to_numeric(data.Minute, errors='coerce')
    data.Second = pd.to_numeric(data.Second, errors='coerce')
    data.Volume = pd.to_numeric(data.Volume, errors='coerce')
    data.Price = pd.to_numeric(data.Price, errors='coerce')

    range_am = pd.date_range("%s-%s-%s 09:00"%(filename[-20:-16],filename[-15:-13],filename[-12:-10]), 
                             "%s-%s-%s 11:30"%(filename[-20:-16],filename[-15:-13],filename[-12:-10]), freq="1min")
    
    range_pm = pd.date_range("%s-%s-%s 12:30"%(filename[-20:-16],filename[-15:-13],filename[-12:-10]), 
                             "%s-%s-%s 15:00"%(filename[-20:-16],filename[-15:-13],filename[-12:-10]), freq="1min")
    
    time = range_am.append(range_pm)

    high=[];low=[];opening=[];closing=[];volume=[];var=[];trans=[]
    for i in range(len(time)):
        tmp_price = list(data.Price[(data['Hour'] == time[i].hour) & (data['Minute'] == time[i].minute)])
        tmp_vol = list(data.Volume[(data['Hour'] == time[i].hour) & (data['Minute'] == time[i].minute)])
        trans.append(len(tmp_price))
        if not tmp_price: 
            high.append(np.nan)
            low.append(np.nan)
            opening.append(np.nan)
            closing.append(np.nan)
            volume.append(np.nansum(tmp_vol))
        else:
            high.append(max(tmp_price))
            low.append(min(tmp_price))
            opening.append(tmp_price[0])
            closing.append(tmp_price[-1])
            volume.append(np.nansum(tmp_vol))

    df = pd.DataFrame({'Time':time, 
                       'High':high, 'Low':low, 'Open':opening, 'Close':closing, 
                       'Volume':volume, 'Transactions':trans})
    ret=[]
    fnna = 0
    while np.isnan(df.Close[fnna]):
        ret.append(df.Close[fnna])
        fnna += 1
    i = fnna
    ret.append(np.nan)
    while i < len(df)-1:
        if not (np.isnan(df.Close[i]) or np.isnan(df.Close[i+1])):
            a = math.log(df.Close[i+1]/df.Close[i])
            ret.append(a)
            i+=1
        else:
            start = df.Close[i]
            counter = 1
            while (i < len(df)-2) and np.isnan(df.Close[i+1]):
                counter += 1
                i+=1
            end = df.Close[i+1]
            a = math.log(end/start)/(counter)
            i+=1
            for _ in range(counter):
                ret.append(a)    
    df['Return']=ret

    vlm_yen = []; ilqd = []
    for i in range(len(df)):
        vlm_yen.append(df.Close[i]*df.Volume[i]/1000000000)   # Billion Yen
    df['Yen Volume'] = vlm_yen
    
    for i in range(len(df)):
        ilqd.append(abs(df.Return[i])/vlm_yen[i])
    df['Illiquidity'] = ilqd
    
    Tmp = df.fillna(0)
    Tmp['Return'][0]=np.nan
    Tmp['Illiquidity'][0]=np.nan
    
    for t in ['05','10','15','30','45','60']:
        tmp_ret = Tmp['Return'].rolling(window=int(t), center=False).apply(np.nansum)
        df['Return-%s'%t]=tmp_ret
        tmp_vlm = Tmp['Volume'].rolling(window=int(t), center=False).apply(np.nansum)
        df['Volume-%s'%t]=tmp_vlm
        tmp_trans = Tmp['Transactions'].rolling(window=int(t), center=False).apply(np.nansum)
        df['Transactions-%s'%t]=tmp_trans
        tmp_yenvlm = Tmp['Yen Volume'].rolling(window=int(t), center=False).apply(np.nansum)
        df['Yen Volume-%s'%t]=tmp_yenvlm
        tmp_ilqd = Tmp['Illiquidity'].rolling(window=int(t), center=False).apply(np.nanmean)
        df['Illiquidity-%s'%t]=tmp_ilqd      
    
    return df

In [3]:
def tick_size(df,filename):
    ts =[]
    
    for i in range(len(df)):
        if float(filename[-20:-16])==2013:
            if df.High[i] <= 3000:
                ts.append(1)
            elif df.High[i] <= 5000:
                ts.append(5)
            elif df.High[i] <= 30000:
                ts.append(10)
            elif df.High[i] <= 50000:
                ts.append(50)
            elif df.High[i] <= 100000:
                ts.append(100)
            else:
                ts.append(np.nan)                
        elif float(filename[-20:-16])==2014:
            if float(filename[-15:-13])==1 and float(filename[-12:-10])<14:
                if df.High[i] <= 3000:
                    ts.append(1)
                elif df.High[i] <= 5000:
                    ts.append(5)
                elif df.High[i] <= 30000:
                    ts.append(10)
                elif df.High[i] <= 50000:
                    ts.append(50)
                elif df.High[i] <= 100000:
                    ts.append(100)
                else:
                    ts.append(np.nan)
            elif float(filename[-15:-13])==1 and float(filename[-12:-10])>=14:
                if df.High[i] <= 10000:
                    ts.append(1)
                elif df.High[i] <= 50000:
                    ts.append(5)
                elif df.High[i] <= 100000:
                    ts.append(10)
                else:
                    ts.append(np.nan)
            elif float(filename[-15:-13])==7 and float(filename[-12:-10])>=22:
                if df.High[i] <= 1000:
                    ts.append(0.1)
                elif df.High[i] <= 5000:
                    ts.append(0.5)
                elif df.High[i] <= 10000:
                    ts.append(1)
                elif df.High[i] <= 50000:
                    ts.append(5)
                elif df.High[i] <= 100000:
                    ts.append(10)
                else:
                    ts.append(np.nan)
            elif float(filename[-15:-13])>7 :
                if df.High[i] <= 1000:
                    ts.append(0.1)
                elif df.High[i] <= 5000:
                    ts.append(0.5)
                elif df.High[i] <= 10000:
                    ts.append(1)
                elif df.High[i] <= 50000:
                    ts.append(5)
                elif df.High[i] <= 100000:
                    ts.append(10)
                else:
                    ts.append(np.nan)
            else:
                if df.High[i] <= 10000:
                    ts.append(1)
                elif df.High[i] <= 50000:
                    ts.append(5)
                elif df.High[i] <= 100000:
                    ts.append(10)
                else:
                    ts.append(np.nan)
    
    df['Tick size']=ts
    
    path = '/Volumes/LaCie/Processed_%s/Time/%s/%s' % (filename[-20:-16],filename[-15:-13],filename[-12:-10])
    if not os.path.exists(path):
        os.makedirs(path)
    pd.DataFrame(df).to_csv('/Volumes/LaCie/Processed_%s/Time/%s/%s/%s.csv'%
                            (filename[-20:-16],filename[-15:-13],filename[-12:-10],filename[-9:-4]), 
                            index=False)

In [4]:
def process (filename):
    df = pre_process(filename)
    tick_size(df,filename)

## Processing Technical Information for 5, 10, 15, 30, 45, 60 Minutes

In [21]:
for y in ['2013','2014']:
    path = '/Volumes/LaCie/Transactions_%s'%y
    files=[]
    for fpathe,dirs,fs in os.walk(path):
        for f in fs:
            if f[-5:]!= 'Store':
                files.append(os.path.join(fpathe,f))
    for i in files:
        print('Processing %s'%i[-20:-4])
        process(i)

Processing 2013/07/01/16050
Processing 2013/07/01/18780
Processing 2013/07/01/19250
Processing 2013/07/01/19280
Processing 2013/07/01/19630
Processing 2013/07/01/25020
Processing 2013/07/01/25030
Processing 2013/07/01/28020
Processing 2013/07/01/29140
Processing 2013/07/01/33820
Processing 2013/07/01/34020
Processing 2013/07/01/34070
Processing 2013/07/01/40630
Processing 2013/07/01/41880
Processing 2013/07/01/44520
Processing 2013/07/01/45020
Processing 2013/07/01/45030
Processing 2013/07/01/45230
Processing 2013/07/01/45680
Processing 2013/07/01/45780
Processing 2013/07/01/46610
Processing 2013/07/01/49010
Processing 2013/07/01/49110
Processing 2013/07/01/50200
Processing 2013/07/01/51080
Processing 2013/07/01/54010
Processing 2013/07/01/54110
Processing 2013/07/01/57130
Processing 2013/07/01/58020
Processing 2013/07/01/62730
Processing 2013/07/01/63010
Processing 2013/07/01/63260
Processing 2013/07/01/63670
Processing 2013/07/01/65010
Processing 2013/07/01/65020
Processing 2013/07/0

Processing 2013/07/03/95320
Processing 2013/07/03/97350
Processing 2013/07/03/99830
Processing 2013/07/03/99840
Processing 2013/07/04/16050
Processing 2013/07/04/18780
Processing 2013/07/04/19250
Processing 2013/07/04/19280
Processing 2013/07/04/19630
Processing 2013/07/04/25020
Processing 2013/07/04/25030
Processing 2013/07/04/28020
Processing 2013/07/04/29140
Processing 2013/07/04/33820
Processing 2013/07/04/34020
Processing 2013/07/04/34070
Processing 2013/07/04/40630
Processing 2013/07/04/41880
Processing 2013/07/04/44520
Processing 2013/07/04/45020
Processing 2013/07/04/45030
Processing 2013/07/04/45230
Processing 2013/07/04/45680
Processing 2013/07/04/45780
Processing 2013/07/04/46610
Processing 2013/07/04/49010
Processing 2013/07/04/49110
Processing 2013/07/04/50200
Processing 2013/07/04/51080
Processing 2013/07/04/54010
Processing 2013/07/04/54110
Processing 2013/07/04/57130
Processing 2013/07/04/58020
Processing 2013/07/04/62730
Processing 2013/07/04/63010
Processing 2013/07/0

Processing 2013/07/08/94370
Processing 2013/07/08/95020
Processing 2013/07/08/95030
Processing 2013/07/08/95310
Processing 2013/07/08/95320
Processing 2013/07/08/97350
Processing 2013/07/08/99830
Processing 2013/07/08/99840
Processing 2013/07/09/16050
Processing 2013/07/09/18780
Processing 2013/07/09/19250
Processing 2013/07/09/19280
Processing 2013/07/09/19630
Processing 2013/07/09/25020
Processing 2013/07/09/25030
Processing 2013/07/09/28020
Processing 2013/07/09/29140
Processing 2013/07/09/33820
Processing 2013/07/09/34020
Processing 2013/07/09/34070
Processing 2013/07/09/40630
Processing 2013/07/09/41880
Processing 2013/07/09/44520
Processing 2013/07/09/45020
Processing 2013/07/09/45030
Processing 2013/07/09/45230
Processing 2013/07/09/45680
Processing 2013/07/09/45780
Processing 2013/07/09/46610
Processing 2013/07/09/49010
Processing 2013/07/09/49110
Processing 2013/07/09/50200
Processing 2013/07/09/51080
Processing 2013/07/09/54010
Processing 2013/07/09/54110
Processing 2013/07/0

Processing 2013/07/11/90640
Processing 2013/07/11/92020
Processing 2013/07/11/94320
Processing 2013/07/11/94330
Processing 2013/07/11/94370
Processing 2013/07/11/95020
Processing 2013/07/11/95030
Processing 2013/07/11/95310
Processing 2013/07/11/95320
Processing 2013/07/11/97350
Processing 2013/07/11/99830
Processing 2013/07/11/99840
Processing 2013/07/12/16050
Processing 2013/07/12/18780
Processing 2013/07/12/19250
Processing 2013/07/12/19280
Processing 2013/07/12/19630
Processing 2013/07/12/25020
Processing 2013/07/12/25030
Processing 2013/07/12/28020
Processing 2013/07/12/29140
Processing 2013/07/12/33820
Processing 2013/07/12/34020
Processing 2013/07/12/34070
Processing 2013/07/12/40630
Processing 2013/07/12/41880
Processing 2013/07/12/44520
Processing 2013/07/12/45020
Processing 2013/07/12/45030
Processing 2013/07/12/45230
Processing 2013/07/12/45680
Processing 2013/07/12/45780
Processing 2013/07/12/46610
Processing 2013/07/12/49010
Processing 2013/07/12/49110
Processing 2013/07/1

Processing 2013/07/17/88010
Processing 2013/07/17/88020
Processing 2013/07/17/88300
Processing 2013/07/17/90200
Processing 2013/07/17/90210
Processing 2013/07/17/90220
Processing 2013/07/17/90640
Processing 2013/07/17/92020
Processing 2013/07/17/94320
Processing 2013/07/17/94330
Processing 2013/07/17/94370
Processing 2013/07/17/95020
Processing 2013/07/17/95030
Processing 2013/07/17/95310
Processing 2013/07/17/95320
Processing 2013/07/17/97350
Processing 2013/07/17/99830
Processing 2013/07/17/99840
Processing 2013/07/18/16050
Processing 2013/07/18/18780
Processing 2013/07/18/19250
Processing 2013/07/18/19280
Processing 2013/07/18/19630
Processing 2013/07/18/25020
Processing 2013/07/18/25030
Processing 2013/07/18/28020
Processing 2013/07/18/29140
Processing 2013/07/18/33820
Processing 2013/07/18/34020
Processing 2013/07/18/34070
Processing 2013/07/18/40630
Processing 2013/07/18/41880
Processing 2013/07/18/44520
Processing 2013/07/18/45020
Processing 2013/07/18/45030
Processing 2013/07/1

...

Processing 2014/12/09/80350
Processing 2014/12/09/80530
Processing 2014/12/09/80580
Processing 2014/12/09/81130
Processing 2014/12/09/82670
Processing 2014/12/09/83060
Processing 2014/12/09/83080
Processing 2014/12/09/83090
Processing 2014/12/09/83160
Processing 2014/12/09/83320
Processing 2014/12/09/84110
Processing 2014/12/09/85910
Processing 2014/12/09/86010
Processing 2014/12/09/86040
Processing 2014/12/09/86300
Processing 2014/12/09/87250
Processing 2014/12/09/87500
Processing 2014/12/09/87660
Processing 2014/12/09/87950
Processing 2014/12/09/88010
Processing 2014/12/09/88020
Processing 2014/12/09/88300
Processing 2014/12/09/90200
Processing 2014/12/09/90210
Processing 2014/12/09/90220
Processing 2014/12/09/90640
Processing 2014/12/09/92020
Processing 2014/12/09/94320
Processing 2014/12/09/94330
Processing 2014/12/09/94370
Processing 2014/12/09/95020
Processing 2014/12/09/95030
Processing 2014/12/09/95310
Processing 2014/12/09/95320
Processing 2014/12/09/97350
Processing 2014/12/0

Processing 2014/12/12/77510
Processing 2014/12/12/77520
Processing 2014/12/12/79120
Processing 2014/12/12/79740
Processing 2014/12/12/80010
Processing 2014/12/12/80020
Processing 2014/12/12/80310
Processing 2014/12/12/80350
Processing 2014/12/12/80530
Processing 2014/12/12/80580
Processing 2014/12/12/81130
Processing 2014/12/12/82670
Processing 2014/12/12/83060
Processing 2014/12/12/83080
Processing 2014/12/12/83090
Processing 2014/12/12/83160
Processing 2014/12/12/83320
Processing 2014/12/12/84110
Processing 2014/12/12/85910
Processing 2014/12/12/86010
Processing 2014/12/12/86040
Processing 2014/12/12/86300
Processing 2014/12/12/87250
Processing 2014/12/12/87500
Processing 2014/12/12/87660
Processing 2014/12/12/87950
Processing 2014/12/12/88010
Processing 2014/12/12/88020
Processing 2014/12/12/88300
Processing 2014/12/12/90200
Processing 2014/12/12/90210
Processing 2014/12/12/90220
Processing 2014/12/12/90640
Processing 2014/12/12/92020
Processing 2014/12/12/94320
Processing 2014/12/1

Processing 2014/12/17/72020
Processing 2014/12/17/72030
Processing 2014/12/17/72610
Processing 2014/12/17/72670
Processing 2014/12/17/72690
Processing 2014/12/17/72700
Processing 2014/12/17/77410
Processing 2014/12/17/77510
Processing 2014/12/17/77520
Processing 2014/12/17/79120
Processing 2014/12/17/79740
Processing 2014/12/17/80010
Processing 2014/12/17/80020
Processing 2014/12/17/80310
Processing 2014/12/17/80350
Processing 2014/12/17/80530
Processing 2014/12/17/80580
Processing 2014/12/17/81130
Processing 2014/12/17/82670
Processing 2014/12/17/83060
Processing 2014/12/17/83080
Processing 2014/12/17/83090
Processing 2014/12/17/83160
Processing 2014/12/17/83320
Processing 2014/12/17/84110
Processing 2014/12/17/85910
Processing 2014/12/17/86010
Processing 2014/12/17/86040
Processing 2014/12/17/86300
Processing 2014/12/17/87250
Processing 2014/12/17/87500
Processing 2014/12/17/87660
Processing 2014/12/17/87950
Processing 2014/12/17/88010
Processing 2014/12/17/88020
Processing 2014/12/1

Processing 2014/12/22/69020
Processing 2014/12/22/69540
Processing 2014/12/22/69710
Processing 2014/12/22/69810
Processing 2014/12/22/69880
Processing 2014/12/22/70110
Processing 2014/12/22/72010
Processing 2014/12/22/72020
Processing 2014/12/22/72030
Processing 2014/12/22/72610
Processing 2014/12/22/72670
Processing 2014/12/22/72690
Processing 2014/12/22/72700
Processing 2014/12/22/77410
Processing 2014/12/22/77510
Processing 2014/12/22/77520
Processing 2014/12/22/79120
Processing 2014/12/22/79740
Processing 2014/12/22/80010
Processing 2014/12/22/80020
Processing 2014/12/22/80310
Processing 2014/12/22/80350
Processing 2014/12/22/80530
Processing 2014/12/22/80580
Processing 2014/12/22/81130
Processing 2014/12/22/82670
Processing 2014/12/22/83060
Processing 2014/12/22/83080
Processing 2014/12/22/83090
Processing 2014/12/22/83160
Processing 2014/12/22/83320
Processing 2014/12/22/84110
Processing 2014/12/22/85910
Processing 2014/12/22/86010
Processing 2014/12/22/86040
Processing 2014/12/2

Processing 2014/12/26/65020
Processing 2014/12/26/65030
Processing 2014/12/26/65940
Processing 2014/12/26/67020
Processing 2014/12/26/67520
Processing 2014/12/26/67580
Processing 2014/12/26/68610
Processing 2014/12/26/69020
Processing 2014/12/26/69540
Processing 2014/12/26/69710
Processing 2014/12/26/69810
Processing 2014/12/26/69880
Processing 2014/12/26/70110
Processing 2014/12/26/72010
Processing 2014/12/26/72020
Processing 2014/12/26/72030
Processing 2014/12/26/72610
Processing 2014/12/26/72670
Processing 2014/12/26/72690
Processing 2014/12/26/72700
Processing 2014/12/26/77410
Processing 2014/12/26/77510
Processing 2014/12/26/77520
Processing 2014/12/26/79120
Processing 2014/12/26/79740
Processing 2014/12/26/80010
Processing 2014/12/26/80020
Processing 2014/12/26/80310
Processing 2014/12/26/80350
Processing 2014/12/26/80530
Processing 2014/12/26/80580
Processing 2014/12/26/81130
Processing 2014/12/26/82670
Processing 2014/12/26/83060
Processing 2014/12/26/83080
Processing 2014/12/2

# Votatilities

## Garman Klass Volatility 
The Garman and Klass estimator for estimating historical volatility assumes Brownian motion with zero drift and no opening jumps (i.e. the opening = close of the previous period). This estimator is 7.4 times more efficient than the close-to-close estimator. 

\begin{align*}
\sigma_{gk} = \sqrt{ \frac{N}{n} \sum \left[ \textstyle\frac{1}{2}\displaystyle \left( \ln \frac{H_i}{L_i} \right)^2  - (2\ln 2-1)  \left( \ln \frac{C_i}{O_i} \right)^2 \right]}
\end{align*}



## Yang Zhang Volatility
Yang and Zhang were the first to derive an historical volatility estimator that has a minimum estimation error, is independent of the drift, and independent of opening gaps. This estimator is maximally 14 times more efficient than the close-to-close estimator. 
It can be interpreted as a weighted average of the Rogers and Satchell estimator, the close-open volatility and the open-close volatility. 
The performance degrades to the classic close-to-close estimator when the price process is heavily dominated by opening jumps. 

\begin{align*}
\sigma_{yz} &= \sqrt{ \sigma_o^2 + k\sigma_c^2 + (1-k)\sigma_{rs}^2} \\ \\
\sigma_o^2 &=\textstyle \frac{N}{n-1} \sum\left( \ln\frac{O_i}{C_{i-1}}-\mu_o \right)^2\\ \\
&\mu_o=\textstyle \frac{1}{n} \sum \ln\frac{O_i}{C_{i-1}}\\ \\
\sigma_c^2 &=\textstyle \frac{N}{n-1} \sum\left( \ln\frac{C_i}{O_i}-\mu_c \right)^2\\ \\
&\mu_c=\textstyle \frac{1}{n} \sum \ln\frac{C_i}{O_i}\\ \\
\sigma_{rs}^2 &= \textstyle\frac{N}{n} \sum \left( 
 \ln \textstyle\frac{H_i}{C_i}    \ln \textstyle\frac{H_i}{O_i} + 
 \ln \textstyle\frac{L_i}{C_i}    \ln \textstyle\frac{L_i}{O_i} 
 \right) \\ \\
k&=\frac{0.34}{1+\frac{n+1}{n-1}}
\end{align*}



## Processing Volatilities for 5, 10, 15, 30, 45, 60 Minutes

In [28]:
def _yz(time,N,df):
    
    n = int(time)
    
    df.High = pd.to_numeric(df.High, errors='coerce')
    df.Low = pd.to_numeric(df.Low, errors='coerce')
    df.Open = pd.to_numeric(df.Open, errors='coerce')
    df.Close = pd.to_numeric(df.Close, errors='coerce')

    k = 0.34/(1+(n+1)/(n-1))
    
    log_Oc = (df['Open'] / df['Close'].shift(1)).apply(np.log)

    def f1(v):
        return (statistics.variance(v))*N
    
    var_overnight =  pd.DataFrame(log_Oc).rolling(window=n, center=False).apply(func=f1)
    
    log_CO = (df['Close'] /df['Open']).apply(np.log)
    var_cl2op = pd.DataFrame(log_CO).rolling(window=n, center=False).apply(func=f1)
    
    log_HC = (df['High'] / df['Close']).apply(np.log)
    log_HO = (df['High'] / df['Open']).apply(np.log)
    log_LC = (df['Low'] / df['Close']).apply(np.log)
    log_LO = (df['Low'] / df['Open']).apply(np.log)
    
    log_rs = log_HC*log_HO + log_LC*log_LO
    var_rs = pd.DataFrame(log_rs).rolling(window=n, center=False).mean()*N
    
    vol_yz = (var_overnight + k*var_cl2op + (1-k)*var_rs)**0.5
    
    return vol_yz

In [30]:
def _gk(time,N,df):
    n = int(time)
    df.High = pd.to_numeric(df.High, errors='coerce')
    df.Low = pd.to_numeric(df.Low, errors='coerce')
    df.Open = pd.to_numeric(df.Open, errors='coerce')
    df.Close = pd.to_numeric(df.Close, errors='coerce')

    log_HL_sq = ((df['High'] / df['Low']).apply(np.log))**2
    log_CO_sq = ((df['Close'] / df['Open']).apply(np.log))**2
    
    tmp = 0.5*log_HL_sq - (2*np.log(2)-1)*log_CO_sq
    def f2(v):
        return (v.mean()*N)**0.5
    
    vol_gk = pd.DataFrame(tmp).rolling(window=n, center=False).apply(func=f2)
    
    return vol_gk

In [31]:
def _cc(time,N,df):
    n = int(time)
    df.Return = pd.to_numeric(df.Return, errors='coerce')
        
    def f3(v):
        return statistics.stdev(v)*(N)**0.5
    
    vol_cc = pd.DataFrame(df.Return).rolling(window=n-1, center=False).apply(func=f3)
    
    return vol_cc

In [34]:
def vol(filename):
    df = pd.read_csv(filename)[['Time','High','Low','Open','Close','Return']]
    N = 300
    for i in ['05','10','15','30','45','60']:
        vol_yz = _yz(i,N,df)   
        vol_gk = _gk(i,N,df)
        vol_cc = _cc(i,N,df)
        df['YZ Volatility-%s'%i] = vol_yz
        df['GK Volatility-%s'%i] = vol_gk
        df['CC Volatility-%s'%i] = vol_cc

    path = '/Volumes/LaCie/Processed_%s/Vola/%s/%s' % (filename[-25:-21],filename[-15:-13],
                                             filename[-12:-10])
    if not os.path.exists(path):
        os.makedirs(path)
    pd.DataFrame(df).to_csv('/Volumes/LaCie/Processed_%s/Vola/%s/%s/%s.csv' %
                             (filename[-25:-21],filename[-15:-13],filename[-12:-10],
                              filename[-9:-4]), index=False)

In [53]:
path = '/Volumes/LaCie/Processed_2014/Time'
files=[]
for fpathe,dirs,fs in os.walk(path):
    for f in fs:
        if f[-5:]!= 'Store':
            files.append(os.path.join(fpathe,f))

In [60]:
%%notify 
for i in files:
    print('Processing %s'%i[-15:-4])
    vol(i)

Processing 09/04/62730
Processing 09/04/63010
Processing 09/04/63260
Processing 09/04/63670
Processing 09/04/65010
Processing 09/04/65020
Processing 09/04/65030
Processing 09/04/65940
Processing 09/04/67020
Processing 09/04/67520
Processing 09/04/67580
Processing 09/04/68610
Processing 09/04/69020
Processing 09/04/69540
Processing 09/04/69710
Processing 09/04/69810
Processing 09/04/69880
Processing 09/04/70110
Processing 09/04/72010
Processing 09/04/72020
Processing 09/04/72030
Processing 09/04/72610
Processing 09/04/72670
Processing 09/04/72690
Processing 09/04/72700
Processing 09/04/77410
Processing 09/04/77510
Processing 09/04/77520
Processing 09/04/79120
Processing 09/04/79740
Processing 09/04/80010
Processing 09/04/80020
Processing 09/04/80310
Processing 09/04/80350
Processing 09/04/80530
Processing 09/04/80580
Processing 09/04/81130
Processing 09/04/82670
Processing 09/04/83060
Processing 09/04/83080
Processing 09/04/83090
Processing 09/04/83160
Processing 09/04/83320
Processing 

Processing 09/09/90220
Processing 09/09/90640
Processing 09/09/92020
Processing 09/09/94320
Processing 09/09/94330
Processing 09/09/94370
Processing 09/09/95020
Processing 09/09/95030
Processing 09/09/95310
Processing 09/09/95320
Processing 09/09/97350
Processing 09/09/99830
Processing 09/09/99840
Processing 09/10/16050
Processing 09/10/18780
Processing 09/10/19250
Processing 09/10/19280
Processing 09/10/19630
Processing 09/10/25020
Processing 09/10/25030
Processing 09/10/28020
Processing 09/10/29140
Processing 09/10/33820
Processing 09/10/34020
Processing 09/10/34070
Processing 09/10/40630
Processing 09/10/41880
Processing 09/10/44520
Processing 09/10/45020
Processing 09/10/45030
Processing 09/10/45230
Processing 09/10/45680
Processing 09/10/45780
Processing 09/10/46610
Processing 09/10/47550
Processing 09/10/49010
Processing 09/10/49110
Processing 09/10/50200
Processing 09/10/51080
Processing 09/10/54010
Processing 09/10/54110
Processing 09/10/57130
Processing 09/10/58020
Processing 

Processing 09/16/69710
Processing 09/16/69810
Processing 09/16/69880
Processing 09/16/70110
Processing 09/16/72010
Processing 09/16/72020
Processing 09/16/72030
Processing 09/16/72610
Processing 09/16/72670
Processing 09/16/72690
Processing 09/16/72700
Processing 09/16/77410
Processing 09/16/77510
Processing 09/16/77520
Processing 09/16/79120
Processing 09/16/79740
Processing 09/16/80010
Processing 09/16/80020
Processing 09/16/80310
Processing 09/16/80350
Processing 09/16/80530
Processing 09/16/80580
Processing 09/16/81130
Processing 09/16/82670
Processing 09/16/83060
Processing 09/16/83080
Processing 09/16/83090
Processing 09/16/83160
Processing 09/16/83320
Processing 09/16/84110
Processing 09/16/85910
Processing 09/16/86010
Processing 09/16/86040
Processing 09/16/86300
Processing 09/16/87250
Processing 09/16/87500
Processing 09/16/87660
Processing 09/16/87950
Processing 09/16/88010
Processing 09/16/88020
Processing 09/16/88300
Processing 09/16/90200
Processing 09/16/90210
Processing 

Processing 09/22/18780
Processing 09/22/19250
Processing 09/22/19280
Processing 09/22/19630
Processing 09/22/25020
Processing 09/22/25030
Processing 09/22/28020
Processing 09/22/29140
Processing 09/22/33820
Processing 09/22/34020
Processing 09/22/34070
Processing 09/22/40630
Processing 09/22/41880
Processing 09/22/44520
Processing 09/22/45020
Processing 09/22/45030
Processing 09/22/45230
Processing 09/22/45680
Processing 09/22/45780
Processing 09/22/46610
Processing 09/22/47550
Processing 09/22/49010
Processing 09/22/49110
Processing 09/22/50200
Processing 09/22/51080
Processing 09/22/54010
Processing 09/22/54110
Processing 09/22/57130
Processing 09/22/58020
Processing 09/22/62730
Processing 09/22/63010
Processing 09/22/63260
Processing 09/22/63670
Processing 09/22/65010
Processing 09/22/65020
Processing 09/22/65030
Processing 09/22/65940
Processing 09/22/67020
Processing 09/22/67520
Processing 09/22/67580
Processing 09/22/68610
Processing 09/22/69020
Processing 09/22/69540
Processing 

Processing 09/26/79120
Processing 09/26/79740
Processing 09/26/80010
Processing 09/26/80020
Processing 09/26/80310
Processing 09/26/80350
Processing 09/26/80530
Processing 09/26/80580
Processing 09/26/81130
Processing 09/26/82670
Processing 09/26/83060
Processing 09/26/83080
Processing 09/26/83090
Processing 09/26/83160
Processing 09/26/83320
Processing 09/26/84110
Processing 09/26/85910
Processing 09/26/86010
Processing 09/26/86040
Processing 09/26/86300
Processing 09/26/87250
Processing 09/26/87500
Processing 09/26/87660
Processing 09/26/87950
Processing 09/26/88010
Processing 09/26/88020
Processing 09/26/88300
Processing 09/26/90200
Processing 09/26/90210
Processing 09/26/90220
Processing 09/26/90640
Processing 09/26/92020
Processing 09/26/94320
Processing 09/26/94330
Processing 09/26/94370
Processing 09/26/95020
Processing 09/26/95030
Processing 09/26/95310
Processing 09/26/95320
Processing 09/26/97350
Processing 09/26/99830
Processing 09/26/99840
Processing 09/29/16050
Processing 

...

Processing 12/10/77510
Processing 12/10/77520
Processing 12/10/79120
Processing 12/10/79740
Processing 12/10/80010
Processing 12/10/80020
Processing 12/10/80310
Processing 12/10/80350
Processing 12/10/80530
Processing 12/10/80580
Processing 12/10/81130
Processing 12/10/82670
Processing 12/10/83060
Processing 12/10/83080
Processing 12/10/83090
Processing 12/10/83160
Processing 12/10/83320
Processing 12/10/84110
Processing 12/10/85910
Processing 12/10/86010
Processing 12/10/86040
Processing 12/10/86300
Processing 12/10/87250
Processing 12/10/87500
Processing 12/10/87660
Processing 12/10/87950
Processing 12/10/88010
Processing 12/10/88020
Processing 12/10/88300
Processing 12/10/90200
Processing 12/10/90210
Processing 12/10/90220
Processing 12/10/90640
Processing 12/10/92020
Processing 12/10/94320
Processing 12/10/94330
Processing 12/10/94370
Processing 12/10/95020
Processing 12/10/95030
Processing 12/10/95310
Processing 12/10/95320
Processing 12/10/97350
Processing 12/10/99830
Processing 

Processing 12/16/41880
Processing 12/16/44520
Processing 12/16/45020
Processing 12/16/45030
Processing 12/16/45230
Processing 12/16/45680
Processing 12/16/45780
Processing 12/16/46610
Processing 12/16/47550
Processing 12/16/49010
Processing 12/16/49110
Processing 12/16/50200
Processing 12/16/51080
Processing 12/16/54010
Processing 12/16/54110
Processing 12/16/57130
Processing 12/16/58020
Processing 12/16/62730
Processing 12/16/63010
Processing 12/16/63260
Processing 12/16/63670
Processing 12/16/65010
Processing 12/16/65020
Processing 12/16/65030
Processing 12/16/65940
Processing 12/16/67020
Processing 12/16/67520
Processing 12/16/67580
Processing 12/16/68610
Processing 12/16/69020
Processing 12/16/69540
Processing 12/16/69710
Processing 12/16/69810
Processing 12/16/69880
Processing 12/16/70110
Processing 12/16/72010
Processing 12/16/72020
Processing 12/16/72030
Processing 12/16/72610
Processing 12/16/72670
Processing 12/16/72690
Processing 12/16/72700
Processing 12/16/77410
Processing 

Processing 12/19/83090
Processing 12/19/83160
Processing 12/19/83320
Processing 12/19/84110
Processing 12/19/85910
Processing 12/19/86010
Processing 12/19/86040
Processing 12/19/86300
Processing 12/19/87250
Processing 12/19/87500
Processing 12/19/87660
Processing 12/19/87950
Processing 12/19/88010
Processing 12/19/88020
Processing 12/19/88300
Processing 12/19/90200
Processing 12/19/90210
Processing 12/19/90220
Processing 12/19/90640
Processing 12/19/92020
Processing 12/19/94320
Processing 12/19/94330
Processing 12/19/94370
Processing 12/19/95020
Processing 12/19/95030
Processing 12/19/95310
Processing 12/19/95320
Processing 12/19/97350
Processing 12/19/99830
Processing 12/19/99840
Processing 12/22/16050
Processing 12/22/18780
Processing 12/22/19250
Processing 12/22/19280
Processing 12/22/19630
Processing 12/22/25020
Processing 12/22/25030
Processing 12/22/28020
Processing 12/22/29140
Processing 12/22/33820
Processing 12/22/34020
Processing 12/22/34070
Processing 12/22/40630
Processing 

Processing 12/26/54110
Processing 12/26/57130
Processing 12/26/58020
Processing 12/26/62730
Processing 12/26/63010
Processing 12/26/63260
Processing 12/26/63670
Processing 12/26/65010
Processing 12/26/65020
Processing 12/26/65030
Processing 12/26/65940
Processing 12/26/67020
Processing 12/26/67520
Processing 12/26/67580
Processing 12/26/68610
Processing 12/26/69020
Processing 12/26/69540
Processing 12/26/69710
Processing 12/26/69810
Processing 12/26/69880
Processing 12/26/70110
Processing 12/26/72010
Processing 12/26/72020
Processing 12/26/72030
Processing 12/26/72610
Processing 12/26/72670
Processing 12/26/72690
Processing 12/26/72700
Processing 12/26/77410
Processing 12/26/77510
Processing 12/26/77520
Processing 12/26/79120
Processing 12/26/79740
Processing 12/26/80010
Processing 12/26/80020
Processing 12/26/80310
Processing 12/26/80350
Processing 12/26/80530
Processing 12/26/80580
Processing 12/26/81130
Processing 12/26/82670
Processing 12/26/83060
Processing 12/26/83080
Processing 

<IPython.core.display.Javascript object>

In [61]:
path = '/Volumes/LaCie/Processed_2013/Time'
files=[]
for fpathe,dirs,fs in os.walk(path):
    for f in fs:
        if f[-5:]!= 'Store':
            files.append(os.path.join(fpathe,f))

In [62]:
%%notify 
for i in files:
    print('Processing %s'%i[-15:-4])
    vol(i)

Processing 07/01/16050
Processing 07/01/18780
Processing 07/01/19250
Processing 07/01/19280
Processing 07/01/19630
Processing 07/01/25020
Processing 07/01/25030
Processing 07/01/28020
Processing 07/01/29140
Processing 07/01/33820
Processing 07/01/34020
Processing 07/01/34070
Processing 07/01/40630
Processing 07/01/41880
Processing 07/01/44520
Processing 07/01/45020
Processing 07/01/45030
Processing 07/01/45230
Processing 07/01/45680
Processing 07/01/45780
Processing 07/01/46610
Processing 07/01/49010
Processing 07/01/49110
Processing 07/01/50200
Processing 07/01/51080
Processing 07/01/54010
Processing 07/01/54110
Processing 07/01/57130
Processing 07/01/58020
Processing 07/01/62730
Processing 07/01/63010
Processing 07/01/63260
Processing 07/01/63670
Processing 07/01/65010
Processing 07/01/65020
Processing 07/01/65030
Processing 07/01/65940
Processing 07/01/67020
Processing 07/01/67520
Processing 07/01/67580
Processing 07/01/68610
Processing 07/01/69020
Processing 07/01/69540
Processing 

Processing 07/04/80020
Processing 07/04/80310
Processing 07/04/80350
Processing 07/04/80530
Processing 07/04/80580
Processing 07/04/81130
Processing 07/04/82670
Processing 07/04/83060
Processing 07/04/83080
Processing 07/04/83090
Processing 07/04/83160
Processing 07/04/83320
Processing 07/04/84110
Processing 07/04/85910
Processing 07/04/86010
Processing 07/04/86040
Processing 07/04/86300
Processing 07/04/87250
Processing 07/04/87500
Processing 07/04/87660
Processing 07/04/87950
Processing 07/04/88010
Processing 07/04/88020
Processing 07/04/88300
Processing 07/04/90200
Processing 07/04/90210
Processing 07/04/90220
Processing 07/04/90640
Processing 07/04/92020
Processing 07/04/94320
Processing 07/04/94330
Processing 07/04/94370
Processing 07/04/95020
Processing 07/04/95030
Processing 07/04/95310
Processing 07/04/95320
Processing 07/04/97350
Processing 07/04/99830
Processing 07/04/99840
Processing 07/05/16050
Processing 07/05/18780
Processing 07/05/19250
Processing 07/05/19280
Processing 

Processing 07/10/49010
Processing 07/10/49110
Processing 07/10/50200
Processing 07/10/51080
Processing 07/10/54010
Processing 07/10/54110
Processing 07/10/57130
Processing 07/10/58020
Processing 07/10/62730
Processing 07/10/63010
Processing 07/10/63260
Processing 07/10/63670
Processing 07/10/65010
Processing 07/10/65020
Processing 07/10/65030
Processing 07/10/65940
Processing 07/10/67020
Processing 07/10/67520
Processing 07/10/67580
Processing 07/10/68610
Processing 07/10/69020
Processing 07/10/69540
Processing 07/10/69710
Processing 07/10/69810
Processing 07/10/69880
Processing 07/10/70110
Processing 07/10/72010
Processing 07/10/72020
Processing 07/10/72030
Processing 07/10/72610
Processing 07/10/72670
Processing 07/10/72690
Processing 07/10/72700
Processing 07/10/77410
Processing 07/10/77510
Processing 07/10/77520
Processing 07/10/79120
Processing 07/10/79740
Processing 07/10/80010
Processing 07/10/80020
Processing 07/10/80310
Processing 07/10/80350
Processing 07/10/80530
Processing 

Processing 07/16/87950
Processing 07/16/88010
Processing 07/16/88020
Processing 07/16/88300
Processing 07/16/90200
Processing 07/16/90210
Processing 07/16/90220
Processing 07/16/90640
Processing 07/16/92020
Processing 07/16/94320
Processing 07/16/94330
Processing 07/16/94370
Processing 07/16/95020
Processing 07/16/95030
Processing 07/16/95310
Processing 07/16/95320
Processing 07/16/97350
Processing 07/16/99830
Processing 07/16/99840
Processing 07/17/16050
Processing 07/17/18780
Processing 07/17/19250
Processing 07/17/19280
Processing 07/17/19630
Processing 07/17/25020
Processing 07/17/25030
Processing 07/17/28020
Processing 07/17/29140
Processing 07/17/33820
Processing 07/17/34020
Processing 07/17/34070
Processing 07/17/40630
Processing 07/17/41880
Processing 07/17/44520
Processing 07/17/45020
Processing 07/17/45030
Processing 07/17/45230
Processing 07/17/45680
Processing 07/17/45780
Processing 07/17/46610
Processing 07/17/47550
Processing 07/17/49010
Processing 07/17/49110
Processing 

Processing 07/22/67020
Processing 07/22/67520
Processing 07/22/67580
Processing 07/22/68610
Processing 07/22/69020
Processing 07/22/69540
Processing 07/22/69710
Processing 07/22/69810
Processing 07/22/69880
Processing 07/22/70110
Processing 07/22/72010
Processing 07/22/72020
Processing 07/22/72030
Processing 07/22/72610
Processing 07/22/72670
Processing 07/22/72690
Processing 07/22/72700
Processing 07/22/77410
Processing 07/22/77510
Processing 07/22/77520
Processing 07/22/79120
Processing 07/22/79740
Processing 07/22/80010
Processing 07/22/80020
Processing 07/22/80310
Processing 07/22/80350
Processing 07/22/80530
Processing 07/22/80580
Processing 07/22/81130
Processing 07/22/82670
Processing 07/22/83060
Processing 07/22/83080
Processing 07/22/83090
Processing 07/22/83160
Processing 07/22/83320
Processing 07/22/84110
Processing 07/22/85910
Processing 07/22/86010
Processing 07/22/86040
Processing 07/22/86300
Processing 07/22/87250
Processing 07/22/87500
Processing 07/22/87660
Processing 

...

Processing 12/09/86300
Processing 12/09/87250
Processing 12/09/87500
Processing 12/09/87660
Processing 12/09/87950
Processing 12/09/88010
Processing 12/09/88020
Processing 12/09/88300
Processing 12/09/90200
Processing 12/09/90210
Processing 12/09/90220
Processing 12/09/90640
Processing 12/09/92020
Processing 12/09/94320
Processing 12/09/94330
Processing 12/09/94370
Processing 12/09/95020
Processing 12/09/95030
Processing 12/09/95310
Processing 12/09/95320
Processing 12/09/97350
Processing 12/09/99830
Processing 12/09/99840
Processing 12/10/16050
Processing 12/10/18780
Processing 12/10/19250
Processing 12/10/19280
Processing 12/10/19630
Processing 12/10/25020
Processing 12/10/25030
Processing 12/10/28020
Processing 12/10/29140
Processing 12/10/33820
Processing 12/10/34020
Processing 12/10/34070
Processing 12/10/40630
Processing 12/10/41880
Processing 12/10/44520
Processing 12/10/45020
Processing 12/10/45030
Processing 12/10/45230
Processing 12/10/45680
Processing 12/10/45780
Processing 

Processing 12/13/65010
Processing 12/13/65020
Processing 12/13/65030
Processing 12/13/65940
Processing 12/13/67020
Processing 12/13/67520
Processing 12/13/67580
Processing 12/13/68610
Processing 12/13/69020
Processing 12/13/69540
Processing 12/13/69710
Processing 12/13/69810
Processing 12/13/69880
Processing 12/13/70110
Processing 12/13/72010
Processing 12/13/72020
Processing 12/13/72030
Processing 12/13/72610
Processing 12/13/72670
Processing 12/13/72690
Processing 12/13/72700
Processing 12/13/77410
Processing 12/13/77510
Processing 12/13/77520
Processing 12/13/79120
Processing 12/13/79740
Processing 12/13/80010
Processing 12/13/80020
Processing 12/13/80310
Processing 12/13/80350
Processing 12/13/80530
Processing 12/13/80580
Processing 12/13/81130
Processing 12/13/82670
Processing 12/13/83060
Processing 12/13/83080
Processing 12/13/83090
Processing 12/13/83160
Processing 12/13/83320
Processing 12/13/84110
Processing 12/13/85910
Processing 12/13/86010
Processing 12/13/86040
Processing 

Processing 12/18/94330
Processing 12/18/94370
Processing 12/18/95020
Processing 12/18/95030
Processing 12/18/95310
Processing 12/18/95320
Processing 12/18/97350
Processing 12/18/99830
Processing 12/18/99840
Processing 12/19/16050
Processing 12/19/18780
Processing 12/19/19250
Processing 12/19/19280
Processing 12/19/19630
Processing 12/19/25020
Processing 12/19/25030
Processing 12/19/28020
Processing 12/19/29140
Processing 12/19/33820
Processing 12/19/34020
Processing 12/19/34070
Processing 12/19/40630
Processing 12/19/41880
Processing 12/19/44520
Processing 12/19/45020
Processing 12/19/45030
Processing 12/19/45230
Processing 12/19/45680
Processing 12/19/45780
Processing 12/19/46610
Processing 12/19/47550
Processing 12/19/49010
Processing 12/19/49110
Processing 12/19/50200
Processing 12/19/51080
Processing 12/19/54010
Processing 12/19/54110
Processing 12/19/57130
Processing 12/19/58020
Processing 12/19/62730
Processing 12/19/63010
Processing 12/19/63260
Processing 12/19/63670
Processing 

Processing 12/25/72010
Processing 12/25/72020
Processing 12/25/72030
Processing 12/25/72610
Processing 12/25/72670
Processing 12/25/72690
Processing 12/25/72700
Processing 12/25/77410
Processing 12/25/77510
Processing 12/25/77520
Processing 12/25/79120
Processing 12/25/79740
Processing 12/25/80010
Processing 12/25/80020
Processing 12/25/80310
Processing 12/25/80350
Processing 12/25/80530
Processing 12/25/80580
Processing 12/25/81130
Processing 12/25/82670
Processing 12/25/83060
Processing 12/25/83080
Processing 12/25/83090
Processing 12/25/83160
Processing 12/25/83320
Processing 12/25/84110
Processing 12/25/85910
Processing 12/25/86010
Processing 12/25/86040
Processing 12/25/86300
Processing 12/25/87250
Processing 12/25/87500
Processing 12/25/87660
Processing 12/25/87950
Processing 12/25/88010
Processing 12/25/88020
Processing 12/25/88300
Processing 12/25/90200
Processing 12/25/90210
Processing 12/25/90220
Processing 12/25/90640
Processing 12/25/92020
Processing 12/25/94320
Processing 

<IPython.core.display.Javascript object>